In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install wandb
!pip install transformers==4.0.0
!pip install catalyst==20.11

In [3]:
!wandb login

wandb: Currently logged in as: dimaorekhov (use `wandb login --relogin` to force relogin)


In [4]:
!git clone https://github.com/lehgtrung/egfr-att

fatal: destination path 'egfr-att' already exists and is not an empty directory.


In [5]:
from pathlib import Path
import json
from transformers import AutoTokenizer, BertForSequenceClassification, BertConfig
import pandas as pd
from dataclasses import dataclass
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from catalyst import dl
from catalyst.utils import set_global_seed


ORIGINAL_PAPER_PATH = Path("egfr-att")
import sys
sys.path.append(ORIGINAL_PAPER_PATH.as_posix())


from egfr.dataset import EGFRDataset


DEVICE = torch.device('cuda')


SEED = 21
set_global_seed(SEED)


DATA_PATH = ORIGINAL_PAPER_PATH / "egfr/data/egfr_10_full_ft_pd_lines.json"

In [6]:
EXPERIMENT_NAME = 'transformer-no-descriptor'


@dataclass
class Config:

    tokenizer_path: str = "seyonec/PubChem10M_SMILES_BPE_450k"

    hidden_size: int = 512
    num_hidden_layers: int = 2
    num_attention_heads: int = 8
    intermediate_size: int = 2048
    hidden_dropout_prob: float = 0.1
    attention_probs_dropout_prob: float = 0.1

    batch_size: int = 16
    accumulation_steps: int = 8
  
    num_epochs: int = 100
    patience: int = 10

    scheduler: str = 'OneCycleLR'
    max_lr: float = 0.0005
    warmup_prop: float = 0.2

    logdir: str = f'drive/MyDrive/logdir_{EXPERIMENT_NAME}'


config = Config()

In [7]:
tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_path)


def get_tokenizer_info(tokenizer):
    for key, value in tokenizer.special_tokens_map.items():
        print(f"{key}:", value, getattr(tokenizer, f"{key}_id"))

get_tokenizer_info(tokenizer)


PAD_TOKEN_ID = tokenizer.pad_token_id

bos_token: <s> 0
eos_token: </s> 2
unk_token: <unk> 3
sep_token: </s> 2
pad_token: <pad> 1
cls_token: <s> 0
mask_token: <mask> 4


In [8]:
model_config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=config.hidden_size,
    num_hidden_layers=config.num_hidden_layers,
    num_attention_heads=config.num_attention_heads,
    intermediate_size=config.intermediate_size,
    hidden_dropout_prob=config.hidden_dropout_prob,
    attention_probs_dropout_prob=config.attention_probs_dropout_prob,
    pad_token_id=PAD_TOKEN_ID
)
model = BertForSequenceClassification(config=model_config)

In [10]:
class SequenceEGFRDataset(EGFRDataset):

    def __init__(self, data, tokenizer):
        super().__init__(data, infer=True)
        self.tokenizer = tokenizer
        self.pad_token_id = tokenizer.pad_token_id
        self.encode_smiles()

        self.mord_ft = torch.FloatTensor(self.mord_ft)
        self.non_mord_ft = torch.FloatTensor(self.non_mord_ft)
        self.label = torch.LongTensor(self.label)

    def encode_smiles(self):
        self.smiles = [
            torch.LongTensor(self.tokenizer.encode(s))
            for s in self.smiles
        ]

    def collate_fn(self, batch):
        smiles, mord_ft, non_mord_ft, labels = zip(*batch)
        smiles = pad_sequence(
            smiles, batch_first=True, padding_value=self.pad_token_id
        )
        mord_ft = torch.stack(mord_ft)
        non_mord_ft = torch.stack(non_mord_ft)
        labels = torch.stack(labels)
        return smiles, mord_ft, non_mord_ft, labels

    def make_loader(self, *args, **kwargs):
        return DataLoader(self, *args, collate_fn=self.collate_fn, **kwargs)


In [11]:
train, valid = train_test_split(
    pd.read_json(DATA_PATH, lines=True), test_size=0.2, random_state=42 #  42 hard code is from original paper code 
)


train_dataset = SequenceEGFRDataset(train, tokenizer)
valid_dataset = SequenceEGFRDataset(valid, tokenizer)

In [12]:
print('Max train smiles length:', max(len(s) for s in train_dataset.smiles))
print('Max valid smiles length:', max(len(s) for s in valid_dataset.smiles))

Max train smiles length: 100
Max valid smiles length: 93


In [13]:
loaders = {
    'train': train_dataset.make_loader(batch_size=config.batch_size, shuffle=True),
    'valid': valid_dataset.make_loader(batch_size=config.batch_size)
}

In [14]:
def init_scheduler(
    optimizer: torch.optim.Optimizer,
    num_steps_per_epoch: int,
    config: Config
):

    if config.scheduler == 'OneCycleLR':
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr=config.max_lr,
            epochs=config.num_epochs,
            steps_per_epoch=num_steps_per_epoch,
            pct_start=config.warmup_prop
        )
        return scheduler, 'batch'

    return None, None


In [15]:
optimizer = torch.optim.Adam(model.parameters())

callbacks = [
    dl.OptimizerCallback(accumulation_steps=config.accumulation_steps),
    dl.EarlyStoppingCallback(patience=config.patience),
    dl.WandbLogger(
        project='egfr-project',
        entity='dimaorekhov',
        group='transformer-no-descriptor',
        name=EXPERIMENT_NAME,
        config=config.__dict__
    ),
    dl.AUCCallback()
]

scheduler, mode = init_scheduler(optimizer, len(loaders['train']), config)
if scheduler is not None:
    callbacks.append(dl.SchedulerCallback(mode=mode))

In [16]:
class EgfrNoDescriptorRunner(dl.Runner):

    def _handle_batch(self, batch):
        smiles, _, _, labels = batch
        out = self.model(input_ids=smiles, labels=labels)
        self.batch_metrics['loss'] = out.loss
        self.input = {'targets': labels}
        self.output = {'logits': out.logits}


In [17]:
# be careful not to override log dir
Path(config.logdir).mkdir(exist_ok=True)

In [18]:
runner = EgfrNoDescriptorRunner(device=DEVICE)
runner.train(
    model=model,
    loaders=loaders,
    optimizer=optimizer,
    scheduler=scheduler,        
    num_epochs=config.num_epochs,
    verbose=True,
    logdir=config.logdir,
    callbacks=callbacks
)

wandb: Currently logged in as: dimaorekhov (use `wandb login --relogin` to force relogin)


1/100 * Epoch (train):   3% 6/175 [00:00<00:20,  8.20it/s, loss=0.721, lr=2.000e-05, momentum=0.950]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning:

Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:1241: UserWarning:

To get the last learning rate computed by the scheduler, please use `get_last_lr()`.



1/100 * Epoch (train): 100% 175/175 [00:03<00:00, 48.81it/s, loss=0.363, lr=2.296e-05, momentum=0.949]
1/100 * Epoch (valid): 100% 44/44 [00:00<00:00, 107.24it/s, loss=0.312]
[2020-12-11 22:19:32,313] 
1/100 * Epoch 1 (_base): lr=2.296e-05 | momentum=0.9494
1/100 * Epoch 1 (train): auc/class_00=0.5180 | auc/class_01=0.5148 | auc/mean=0.5164 | loss=0.4682 | lr=2.099e-05 | momentum=0.9498
1/100 * Epoch 1 (valid): auc/class_00=0.4656 | auc/class_01=0.4321 | auc/mean=0.4489 | loss=0.4495
2/100 * Epoch (train): 100% 175/175 [00:03<00:00, 50.56it/s, loss=0.382, lr=3.175e-05, momentum=0.948]
2/100 * Epoch (valid): 100% 44/44 [00:00<00:00, 117.76it/s, loss=0.337]
[2020-12-11 22:19:38,532] 
2/100 * Epoch 2 (_base): lr=3.175e-05 | momentum=0.9476
2/100 * Epoch 2 (train): auc/class_00=0.5099 | auc/class_01=0.5218 | auc/mean=0.5158 | loss=0.4261 | lr=2.689e-05 | momentum=0.9486
2/100 * Epoch 2 (valid): auc/class_00=0.5918 | auc/class_01=0.5941 | auc/mean=0.5929 | loss=0.4113
3/100 * Epoch (train):

auc/class_00/train,0.97478
auc/class_01/train,0.97559
auc/mean/train,0.97519
loss/train,0.12054
lr/train,0.00047
momentum/train,0.85502
auc/class_00/valid,0.89433
auc/class_01/valid,0.89523
auc/mean/valid,0.89478
loss/valid,0.28724
lr/_base,0.00047


auc/class_00/train,▁▁▁▁▁▃▅▇▇███████████████████████
auc/class_01/train,▁▁▁▁▁▃▅▇▇███████████████████████
auc/mean/train,▁▁▁▁▁▃▅▇▇███████████████████████
loss/train,█▇▇▇▇▇▆▄▃▂▂▂▂▂▂▂▂▁▁▂▂▂▂▁▁▁▁▁▁▁▁▁
lr/train,▁▁▁▂▂▂▃▃▄▄▅▅▆▆▇▇▇███████████████
momentum/train,███▇▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
auc/class_00/valid,▁▃▅▄▄▆▇███████████▇▇███▇████▇▇██
auc/class_01/valid,▁▃▄▄▄▆▇████████████████▇████▇▇██
auc/mean/valid,▁▃▄▄▄▆▇███████████▇▇███▇████▇▇██
loss/valid,█▇▇▆▆▅▃▁▁▂▂▂▁▁▂▂▂▂▅▃▁▁▂▄▃▂▃▂▂▆▃▂
lr/_base,▁▁▁▂▂▂▃▃▄▄▅▆▆▇▇▇████████████████


Top best models:
drive/MyDrive/logdir_transformer-no-descriptor/checkpoints/train.22.pth	0.2471
